In [1]:
import pandas as pd

data = pd.read_csv('data/^NDX_raw_data.csv')
data.rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [2]:
data = data.iloc[:3524]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3524 entries, 0 to 3523
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    3524 non-null   object 
 1   open    3524 non-null   float64
 2   high    3524 non-null   float64
 3   low     3524 non-null   float64
 4   close   3524 non-null   float64
 5   volume  3524 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 165.3+ KB


### prepare models for close, high, low

In [4]:
import pandas as pd

#data = pd.read_csv('data_sample_three_years.csv')

#data = data.iloc[:, [0,1,2,3,4]]
#data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

#data['date'] = pd.to_datetime(data['timestamp'], unit='s')
#data = data.iloc[:, [0,5,1,2,3,4]]

#data_backup = data.loc[ data.timestamp >= 1675814400 ]
#data = data.loc[ data.timestamp < 1675814400 ]

from talib import RSI
data['rsi_14'] = RSI(data['close'], timeperiod=14)
data['rsi_30'] = RSI(data['close'], timeperiod=30)

from talib import MA, SMA, EMA
data['ma_9'] = MA(data['close'], timeperiod=9)
data['ma_52'] = MA(data['close'], timeperiod=52)

for i in range(1,21):
    data[f'close+{i}'] = data['close'].shift(i)
for i in range(1,21):
    data[f'high+{i}'] = data['high'].shift(i)
for i in range(1,21):
    data[f'low+{i}'] = data['low'].shift(i)
data.dropna(axis=0, inplace=True)

In [5]:
data.shape

(3473, 70)

In [6]:
inputs_close = data.iloc[:, [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]
target_close = data.iloc[:, [4]]

inputs_highs = data.iloc[:, [6,7,8,9,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49]]
target_highs = data.iloc[:, [2]]

inputs_lows = data.iloc[:, [6,7,8,9,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69]]
target_lows = data.iloc[:, [3]]

In [7]:
data.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'rsi_14', 'rsi_30',
       'ma_9', 'ma_52', 'close+1', 'close+2', 'close+3', 'close+4', 'close+5',
       'close+6', 'close+7', 'close+8', 'close+9', 'close+10', 'close+11',
       'close+12', 'close+13', 'close+14', 'close+15', 'close+16', 'close+17',
       'close+18', 'close+19', 'close+20', 'high+1', 'high+2', 'high+3',
       'high+4', 'high+5', 'high+6', 'high+7', 'high+8', 'high+9', 'high+10',
       'high+11', 'high+12', 'high+13', 'high+14', 'high+15', 'high+16',
       'high+17', 'high+18', 'high+19', 'high+20', 'low+1', 'low+2', 'low+3',
       'low+4', 'low+5', 'low+6', 'low+7', 'low+8', 'low+9', 'low+10',
       'low+11', 'low+12', 'low+13', 'low+14', 'low+15', 'low+16', 'low+17',
       'low+18', 'low+19', 'low+20'],
      dtype='object')

In [8]:
for i, name in enumerate(data.columns):
    print(i, name)

0 date
1 open
2 high
3 low
4 close
5 volume
6 rsi_14
7 rsi_30
8 ma_9
9 ma_52
10 close+1
11 close+2
12 close+3
13 close+4
14 close+5
15 close+6
16 close+7
17 close+8
18 close+9
19 close+10
20 close+11
21 close+12
22 close+13
23 close+14
24 close+15
25 close+16
26 close+17
27 close+18
28 close+19
29 close+20
30 high+1
31 high+2
32 high+3
33 high+4
34 high+5
35 high+6
36 high+7
37 high+8
38 high+9
39 high+10
40 high+11
41 high+12
42 high+13
43 high+14
44 high+15
45 high+16
46 high+17
47 high+18
48 high+19
49 high+20
50 low+1
51 low+2
52 low+3
53 low+4
54 low+5
55 low+6
56 low+7
57 low+8
58 low+9
59 low+10
60 low+11
61 low+12
62 low+13
63 low+14
64 low+15
65 low+16
66 low+17
67 low+18
68 low+19
69 low+20


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(inputs_close, target_close, test_size=0.2, random_state=1, shuffle=False)
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(inputs_highs, target_highs, test_size=0.2, random_state=1, shuffle=False)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(inputs_lows, target_lows, test_size=0.2, random_state=1, shuffle=False)

In [10]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler_c = MinMaxScaler()
scaler_c.fit(X_train_c)
scaler_target_c = MinMaxScaler()
scaler_target_c.fit(y_train_c)

scaler_h = MinMaxScaler()
scaler_h.fit(X_train_h)
scaler_target_h = MinMaxScaler()
scaler_target_h.fit(y_train_h)

scaler_l = MinMaxScaler()
scaler_l.fit(X_train_l)
scaler_target_l = MinMaxScaler()
scaler_target_l.fit(y_train_l)

X_train_c_scaled = scaler_c.transform(X_train_c)
X_test_c_scaled = scaler_c.transform(X_test_c)
y_train_c_scaled = scaler_target_c.transform(y_train_c)
y_test_c_scaled = scaler_target_c.transform(y_test_c)

X_train_h_scaled = scaler_h.transform(X_train_h)
X_test_h_scaled = scaler_h.transform(X_test_h)
y_train_h_scaled = scaler_target_h.transform(y_train_h)
y_test_h_scaled = scaler_target_h.transform(y_test_h)

X_train_l_scaled = scaler_l.transform(X_train_l)
X_test_l_scaled = scaler_l.transform(X_test_l)
y_train_l_scaled = scaler_target_l.transform(y_train_l)
y_test_l_scaled = scaler_target_l.transform(y_test_l)

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import xgboost

In [17]:
inputs_close.shape

(3473, 24)

In [19]:
# Define and train the LSTM model
model_c = Sequential([
    Dense(64, activation='relu', input_shape=(24,)),  # Input layer with 10 features
    Dense(64, activation='relu'),                     # Hidden layer with 64 units
    Dense(1)                                          # Output layer with 1 unit for regression
])

# Compile the model
model_c.compile(optimizer='adam', loss='mse')  # Using Mean Squared Error loss for regression

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Train the model
model_c.fit(X_train_c_scaled, y_train_c_scaled, epochs=500, batch_size=32, callbacks=[early_stopping])

Epoch 1/500
87/87 [==============================] - 1s 2ms/step - loss: 0.0020
Epoch 2/500
87/87 [==============================] - 0s 2ms/step - loss: 6.0192e-05
Epoch 3/500
87/87 [==============================] - 0s 2ms/step - loss: 4.7652e-05
Epoch 4/500
87/87 [==============================] - 0s 2ms/step - loss: 4.2604e-05
Epoch 5/500
87/87 [==============================] - 0s 2ms/step - loss: 3.6964e-05
Epoch 6/500
87/87 [==============================] - 0s 2ms/step - loss: 3.6722e-05
Epoch 7/500
87/87 [==============================] - 0s 2ms/step - loss: 3.2189e-05
Epoch 8/500
87/87 [==============================] - 0s 2ms/step - loss: 2.9910e-05
Epoch 9/500
87/87 [==============================] - 0s 2ms/step - loss: 3.6488e-05
Epoch 10/500
87/87 [==============================] - 0s 2ms/step - loss: 3.0485e-05
Epoch 11/500
87/87 [==============================] - 0s 2ms/step - loss: 2.8869e-05
Epoch 12/500
87/87 [==============================] - 0s 2ms/step - loss: 2.63

In [20]:
# Define and train the LSTM model
model_h = Sequential([
    Dense(64, activation='relu', input_shape=(24,)),  # Input layer with 10 features
    Dense(64, activation='relu'),                     # Hidden layer with 64 units
    Dense(1)                                          # Output layer with 1 unit for regression
])

# Compile the model
model_h.compile(optimizer='adam', loss='mse')  # Using Mean Squared Error loss for regression

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Train the model
model_h.fit(X_train_h_scaled, y_train_h_scaled, epochs=500, batch_size=32, callbacks=[early_stopping])

Epoch 1/500
87/87 [==============================] - 1s 2ms/step - loss: 0.0074
Epoch 2/500
87/87 [==============================] - 0s 2ms/step - loss: 1.8114e-04
Epoch 3/500
87/87 [==============================] - 0s 2ms/step - loss: 1.0804e-04
Epoch 4/500
87/87 [==============================] - 0s 2ms/step - loss: 7.1059e-05
Epoch 5/500
87/87 [==============================] - 0s 2ms/step - loss: 5.4307e-05
Epoch 6/500
87/87 [==============================] - 0s 2ms/step - loss: 4.3760e-05
Epoch 7/500
87/87 [==============================] - 0s 2ms/step - loss: 4.6410e-05
Epoch 8/500
87/87 [==============================] - 0s 2ms/step - loss: 3.7901e-05
Epoch 9/500
87/87 [==============================] - 0s 2ms/step - loss: 3.6675e-05
Epoch 10/500
87/87 [==============================] - 0s 2ms/step - loss: 3.3992e-05
Epoch 11/500
87/87 [==============================] - 0s 2ms/step - loss: 3.3179e-05
Epoch 12/500
87/87 [==============================] - 0s 2ms/step - loss: 3.56

In [21]:
# Define and train the LSTM model
model_l = Sequential([
    Dense(64, activation='relu', input_shape=(24,)),  # Input layer with 10 features
    Dense(64, activation='relu'),                     # Hidden layer with 64 units
    Dense(1)                                          # Output layer with 1 unit for regression
])

# Compile the model
model_l.compile(optimizer='adam', loss='mse')  # Using Mean Squared Error loss for regression

# Adding early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Train the model
model_l.fit(X_train_l_scaled, y_train_l_scaled, epochs=500, batch_size=32, callbacks=[early_stopping])

Epoch 1/500
87/87 [==============================] - 1s 2ms/step - loss: 0.0172
Epoch 2/500
87/87 [==============================] - 0s 2ms/step - loss: 2.2113e-04
Epoch 3/500
87/87 [==============================] - 0s 2ms/step - loss: 8.4674e-05
Epoch 4/500
87/87 [==============================] - 0s 2ms/step - loss: 6.5911e-05
Epoch 5/500
87/87 [==============================] - 0s 2ms/step - loss: 5.8011e-05
Epoch 6/500
87/87 [==============================] - 0s 2ms/step - loss: 5.1970e-05
Epoch 7/500
87/87 [==============================] - 0s 2ms/step - loss: 4.8348e-05
Epoch 8/500
87/87 [==============================] - 0s 2ms/step - loss: 4.7112e-05
Epoch 9/500
87/87 [==============================] - 0s 2ms/step - loss: 4.5319e-05
Epoch 10/500
87/87 [==============================] - 0s 2ms/step - loss: 4.3679e-05
Epoch 11/500
87/87 [==============================] - 0s 2ms/step - loss: 4.3410e-05
Epoch 12/500
87/87 [==============================] - 0s 2ms/step - loss: 4.57

### data reading:

In [47]:
import os
import re

directory = 'data'

df_data = dict()

for file in os.listdir(directory):
    symbol_pattern = re.match(r'([^_]+)_', file)
    symbol = symbol_pattern.group(1)
    df_data[symbol] = pd.read_csv(os.path.join(directory, file))
    df_data[symbol].rename(columns={'Date': 'date', 'Open':'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}, inplace=True)

In [48]:
import pandas as pd

for symbol in df_data:
    from talib import RSI
    df_data[symbol]['rsi_14'] = RSI(df_data[symbol]['close'], timeperiod=14)
    df_data[symbol]['rsi_30'] = RSI(df_data[symbol]['close'], timeperiod=30)

    from talib import MA, SMA, EMA
    df_data[symbol]['ma_9'] = MA(df_data[symbol]['close'], timeperiod=9)
    df_data[symbol]['ma_52'] = MA(df_data[symbol]['close'], timeperiod=52)
    
    for i in range(1,21):
        df_data[symbol][f'close+{i}'] = df_data[symbol]['close'].shift(i)
    for i in range(1,21):
        df_data[symbol][f'high+{i}'] = df_data[symbol]['high'].shift(i)
    for i in range(1,21):
        df_data[symbol][f'low+{i}'] = df_data[symbol]['low'].shift(i)
    df_data[symbol].dropna(axis=0, inplace=True)

In [49]:
#df_input = dict()
df_pred = dict()

count = 0

for symbol in df_data:

    count += 1

    scaler_input = MinMaxScaler()
    data_backup_c_scaled = scaler_input.fit_transform(df_data[symbol].iloc[:, [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]])
    scaler_target = MinMaxScaler()
    target_backup_c_scaled = scaler_target.fit_transform(df_data[symbol][['close']])

    print(count, symbol)
    y_pred_c = model_c.predict(data_backup_c_scaled)
    y_pred_c = scaler_target.inverse_transform(y_pred_c)
    y_pred_c = y_pred_c.reshape(1,-1)[0]

    scaler_input = MinMaxScaler()
    data_backup_h_scaled = scaler_input.fit_transform(df_data[symbol].iloc[:, [6,7,8,9,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49]])
    scaler_target = MinMaxScaler()
    target_backup_h_scaled = scaler_target.fit_transform(df_data[symbol][['high']])

    y_pred_h = model_h.predict(data_backup_h_scaled)
    y_pred_h = scaler_target.inverse_transform(y_pred_h)
    y_pred_h = y_pred_h.reshape(1,-1)[0]

    scaler_input = MinMaxScaler()
    data_backup_l_scaled = scaler_input.fit_transform(df_data[symbol].iloc[:, [6,7,8,9,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69]])
    scaler_target = MinMaxScaler()
    target_backup_l_scaled = scaler_target.fit_transform(df_data[symbol][['low']])

    y_pred_l = model_l.predict(data_backup_l_scaled)
    y_pred_l = scaler_target.inverse_transform(y_pred_l)
    y_pred_l = y_pred_l.reshape(1,-1)[0]

    dict_pred = {'pred_c':y_pred_c, 'pred_h':y_pred_h, 'pred_l':y_pred_l}
    df_pred[symbol] = dict_pred

1 VRTX
156/156 [==============================] - 0s 1ms/step
2 MDLZ
156/156 [==============================] - 0s 1ms/step
3 TMO
156/156 [==============================] - 0s 1ms/step
4 UNH
156/156 [==============================] - 0s 2ms/step
5 TJX
156/156 [==============================] - 0s 1ms/step
6 AAPL
156/156 [==============================] - 0s 1ms/step
7 BKNG
156/156 [==============================] - 0s 1ms/step
8 V
123/123 [==============================] - 0s 2ms/step
9 LOW
156/156 [==============================] - 0s 1ms/step
10 PM
123/123 [==============================] - 0s 1ms/step
11 CMCSA
156/156 [==============================] - 0s 2ms/step
12 HD
156/156 [==============================] - 0s 1ms/step
13 GOOG
151/151 [==============================] - 0s 2ms/step
14 AXP
156/156 [==============================] - 0s 2ms/step
15 NVDA
156/156 [==============================] - 0s 1ms/step
16 SBUX
156/156 [==============================] - 0s 2ms/step
17 PG
156/15

In [50]:
count = 0

for symbol in df_data:
    count += 1
    
    df_data[symbol].reset_index(inplace=True)

    from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error

    score_c = mean_absolute_error(df_data[symbol][['close']], df_pred[symbol]['pred_c'])
    r2_c = r2_score(df_data[symbol][['close']], df_pred[symbol]['pred_c'])
    df_data[symbol]['mae_c'] = score_c
    
    score_h = mean_absolute_error(df_data[symbol][['high']], df_pred[symbol]['pred_h'])
    r2_h = r2_score(df_data[symbol][['high']], df_pred[symbol]['pred_h'])
    df_data[symbol]['mae_h'] = score_h
    
    score_l = mean_absolute_error(df_data[symbol][['low']], df_pred[symbol]['pred_l'])
    r2_l = r2_score(df_data[symbol][['low']], df_pred[symbol]['pred_l'])
    df_data[symbol]['mae_l'] = score_l

    print(f'{count} METRICS FOR {symbol}:')
    print(f'score close: {score_c}')
    print(f'score high: {score_h}')
    print(f'score low: {score_l}')

    print(f'- R2 METRICS FOR {symbol}:')
    print(f'r2 close: {r2_c}')
    print(f'r2 high: {r2_h}')
    print(f'r2 low: {r2_l}')

1 METRICS FOR VRTX:
score close: 4.709593955440868
score high: 3.6128676402812885
score low: 5.782121364297947
- R2 METRICS FOR VRTX:
r2 close: 0.9964552825871543
r2 high: 0.9977739190370609
r2 low: 0.994650312213028
2 METRICS FOR MDLZ:
score close: 0.3208561096742708
score high: 0.383893389340052
score low: 0.3465607764490251
- R2 METRICS FOR MDLZ:
r2 close: 0.9979946963061188
r2 high: 0.9978094162096748
r2 low: 0.9978653515116996
3 METRICS FOR TMO:
score close: 9.259894314275503
score high: 6.340354562619277
score low: 8.273424721108723
- R2 METRICS FOR TMO:
r2 close: 0.9955779023331751
r2 high: 0.9980675333654974
r2 low: 0.9965120264052132
4 METRICS FOR UNH:
score close: 6.742848635901294
score high: 4.437256750453672
score low: 5.846504243489299
- R2 METRICS FOR UNH:
r2 close: 0.9966222576215378
r2 high: 0.9986822818317688
r2 low: 0.9975565627085836
5 METRICS FOR TJX:
score close: 1.1355028572162917
score high: 0.9192689611748106
score low: 1.2642100702467236
- R2 METRICS FOR TJX:


In [51]:
count = 0

remaining_columns = ('date','open','high','low','close','volume','rsi_14','ma_9','mae_c','mae_h','mae_l')

for symbol in df_data:
    count += 1

    df_data[symbol] = df_data[symbol][list(remaining_columns)]

    df_data[symbol]['mpe_c'] = (df_data[symbol]['mae_c'].iloc[-1] / df_data[symbol]['close'].iloc[-1]) * 100
    df_data[symbol]['mpe_h'] = (df_data[symbol]['mae_h'].iloc[-1] / df_data[symbol]['high'].iloc[-1]) * 100
    df_data[symbol]['mpe_l'] = (df_data[symbol]['mae_l'].iloc[-1] / df_data[symbol]['low'].iloc[-1]) * 100
    
    y_pred_c_series = pd.Series(df_pred[symbol]['pred_c'].reshape(1,-1)[0])
    y_pred_h_series = pd.Series(df_pred[symbol]['pred_h'].reshape(1,-1)[0])
    y_pred_l_series = pd.Series(df_pred[symbol]['pred_l'].reshape(1,-1)[0])
    df_data[symbol]['pred_close'] = y_pred_c_series
    df_data[symbol]['pred_highs'] = y_pred_h_series
    df_data[symbol]['pred_lows'] = y_pred_l_series
    df_data[symbol].to_excel(f'data_w_pred_m2/{symbol}_w_pred_m2.xlsx')

    print(f'{count} done for {symbol}')

1 done for VRTX
2 done for MDLZ
3 done for TMO
4 done for UNH
5 done for TJX
6 done for AAPL
7 done for BKNG
8 done for V
9 done for LOW
10 done for PM
11 done for CMCSA
12 done for HD
13 done for GOOG
14 done for AXP
15 done for NVDA
16 done for SBUX
17 done for PG
18 done for SPGI
19 done for AMZN
20 done for ORCL
21 done for NKE
22 done for CVX
23 done for SYK
24 done for TMUS
25 done for XOM
26 done for LRCX
27 done for AMAT
28 done for HON
29 done for ABNB
30 done for GE
31 done for TSLA
32 done for MRK
33 done for BSX
34 done for UPS
35 done for LLY
36 done for VZ
37 done for T
38 done for MSFT
39 done for CI
40 done for ETN
41 done for JNJ
42 done for PANW
43 done for MDT
44 done for DE
45 done for AVGO
46 done for ELV
47 done for SHOP
48 done for DHR
49 done for BA
50 done for C
51 done for META
52 done for SCHW
53 done for CB
54 done for MCD
55 done for LMT
56 done for BMY
57 done for QCOM
58 done for ABT
59 done for NOW
60 done for IBM
61 done for ISRG
62 done for CAT
63 done

In [53]:
df_data['AAPL']['pred_close']

0         5.598937
1         4.630877
2         4.896082
3         4.901911
4         3.792028
           ...    
4977    192.953659
4978    192.633286
4979    192.137939
4980    192.910110
4981    191.624405
Name: pred_close, Length: 4982, dtype: float32